In [1]:
# Loading all the libraries needed for running the code chunks below

import json
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException


In [19]:
def parse_movie(link):   
    
    #movie_data = {'title':'','year':'', 'genre':'', 'summary':'', 'starring':'', 'director':'', 'runtime':''}
    movie_data = {'link':'','title':'', 'summary':'', 'year':'', 'genre':'', 'runtime':''}
    driver.get(link)
    driver.implicitly_wait(1)
    html = driver.page_source
    titles = []
    
    movie_data['link'] = link
    
    try:
        button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
        button.click()
    except NoSuchElementException:
        pass
    
    try:
        title = driver.find_element(By.TAG_NAME, 'h1')
        movie_data['title'] = title.text
        titles.append(title.text)
    except NoSuchElementException:
        pass 
    
    try:
        info = driver.find_element(By.CLASS_NAME, 'scoreboard__info')
        info_string = info.text
        
        # split the string into a list based on the commas
        info_list = info_string.split(", ")

        # extract the individual values from the list
        movie_data['year'] = int(info_list[0])
        if len(info_list)>1:
            movie_data['genre'] = info_list[1]
        if len(info_list)>2:
            movie_data['runtime'] = info_list[2]
        
    except NoSuchElementException:
        pass 
   
    try:
        summary = driver.find_element(By.CLASS_NAME, 'media-body')
        summary2 = summary.find_element(By.TAG_NAME, 'p')
        movie_data['summary'] = summary2.text
    except NoSuchElementException:
        pass
    
    
    return movie_data, titles

In [22]:
# Main code chunk

driver = webdriver.Chrome('chromedriver')
#main_link = 'https://www.metacritic.com/movie/'

with open('movie_titles_for_rotten.txt', 'r') as f:
    movie_titles = [line.strip() for line in f]

urls = ["https://www.rottentomatoes.com/m/" + title for title in movie_titles]

start = time.time()

data = []
titles = []
for link in urls:
    data_l, titles_l = parse_movie(link)
    if titles_l == '':
        continue
    data.append(data_l)
    titles.append(titles_l)

end = time.time()
print('Parsing time:', end-start)
driver.close()

with open("movie_data_rotten.json", "w", encoding='utf-8') as jsonfile:
    json.dump(data, jsonfile, ensure_ascii=False, indent = 2)
    
with open('rotten_titles.txt', 'w') as f:
    f.write(str(titles))

Parsing time: 3582.5282909870148


In [ ]:
with open("rotten_titles.csv", "w", newline="", encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(titles)